# Import `Gismo.jl` and other packages

We import `Gismo.jl`


In [ ]:
using Gismo
using Plots

# Constructing a 2D basis

We use the same B-spline basis as in the previous example, with knot vectors:

$\Xi_u = \Xi_v = \{0,0,0,0.25,0.50,0.75,1,1,1\}$


In [ ]:
kv_u = kv_v = KnotVector([0,0,0,0.25,0.50,0.75,1,1,1]);

In [ ]:
tbasis = TensorBSplineBasis(kv_u, kv_v);

# Definining a surface

Similar to defining a curve, we define a matrix of control points to construct a surface. For a basis of size $N$, the coefficient matrix is defined as an $N\times d$ matrix, with $d$ the geometric dimension. Furthermore, we initialize the coefficient matrix with $N\times d$ zeros.

Since our basis is composed of the same knot vector in both directions, the number of functions in each direction is $n=\sqrt{N}$.

In [ ]:
coefficients = zeros(size(tbasis),3);
n = Int(sqrt(size(tbasis)));

Then, we define the control points in a uniformly spaced grid. We use the tensor product with a vector of ones for this:

In [ ]:
x = y = range(0,stop=1.0,length=n);
X = (x' .* ones(n));
Y = (y' .* ones(n))';
coefficients[:,1] = reduce(vcat,X);
coefficients[:,2] = reduce(vcat,Y);

Since the coefficient matrix is initialized with zeros, we modify a few coefficients to displace the points out-of-plane.

In [ ]:
coefficients[12,3] = -1
coefficients[16,3] = 1
coefficients[22,3] = -1
coefficients[26,3] = 10

Having defined the coefficient matrix, we can make a tensor B-splne surface using the basis and the coefficient matrix.

In [ ]:
surf = TensorBSpline(tbasis,coefficients);

# Plotting the 3D surface

As always, we start by creating a set of parametric points. Here, they stored in `x` and `y`, on their turn used to construct a "mesh grid", described by `XX` and `YY`. The grid is used to create a matrix of parametric points to evaluate the surface on. The matrix stores the points as columns.

In [ ]:
N = M = 100;
x = range(0,stop=1,length=N);
y = range(0,stop=1,length=M);
XX = (x' .* ones(M))';
YY = (y' .* ones(N));
pts = vcat(reduce(vcat,XX)',reduce(vcat,YY)');

The surface is simply evaluated using `val`, and the result is properly reshaped into the format of the mesh grid:

In [ ]:
S = copyMatrix(val(surf,pts));
ZZ = reshape(S[3,:],(N,M));

Then, we plot the surface and its control points

In [ ]:
p = plot();
surface!(XX,YY,ZZ)
# scatter!(coefficients[:,1],coefficients[:,2],coefficients[:,3])
display(p)

# Reading a surface from a file

As for the curve example, we can read the surface from XML.

In [ ]:
path = joinpath(@__DIR__,"filedata/sphere.xml");
XMLSurf = Geometry(path);
XMLcoefs = copyMatrix(coefs(XMLSurf));

Using the same plotting commands as before, we can plot the result

In [ ]:
N = M = 100;
x = range(0,stop=1,length=N);
y = range(0,stop=1,length=M);
XX = (x' .* ones(M))';
YY = (y' .* ones(N));
pts = vcat(reduce(vcat,XX)',reduce(vcat,YY)');

S = copyMatrix(val(XMLSurf,pts));
XX = reshape(S[1,:],(N,M));
YY = reshape(S[2,:],(N,M));
ZZ = reshape(S[3,:],(N,M));

p = plot();
surface!(XX,YY,ZZ)
# scatter!(XMLcoefs[:,1],XMLcoefs[:,2],XMLcoefs[:,3])
display(p)